In [1]:
import os

In [2]:
%pwd

'/mnt/e/bgRemoveApp/bg_removal/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/e/bgRemoveApp/bg_removal'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_weights:str
    params_classes:int

In [6]:
from bgremove.constants import*
from bgremove.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        
        create_directories([self.config.artifacts_root])

    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [8]:
%pwd

'/mnt/e/bgRemoveApp/bg_removal'

In [36]:
from tensorflow.keras.optimizers import Adam

In [37]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Add

In [40]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def conv_block(self, inputs, out_ch, rate=1):
        x = Conv2D(out_ch, 3, padding="same", dilation_rate=1)(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x
    
    def RSU_L(self, inputs, out_ch, int_ch, num_layers, rate=2):
#         initial Conv
        x = self.conv_block(inputs, out_ch)
        init_features = x
        
#         Encoders
        skip = []
        x = self.conv_block(x, int_ch)
        skip.append(x)
        
        for i in range(num_layers-2):
            x = MaxPool2D((2,2))(x)
            x = self.conv_block(x, int_ch)
            skip.append(x)

#         bridge
        x = self.conv_block(x, int_ch, rate=rate)
    
#         decoder
        skip.reverse()
        
        x = Concatenate()([x, skip[0]])
        x = self.conv_block(x, int_ch)

        for i in range(num_layers-3):
            x = UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
            x = Concatenate()([x, skip[i+1]])
            x = self.conv_block(x, int_ch)

        x = UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip[-1]])
        x = self.conv_block(x, out_ch)

        """ Add """
        x = Add()([x, init_features])
        return x
    

    def RSU_4F(self, inputs, out_ch, int_ch):
        """ Initial Conv """
        x0 = self.conv_block(inputs, out_ch, rate=1)

        """ Encoder """
        x1 = self.conv_block(x0, int_ch, rate=1)
        x2 = self.conv_block(x1, int_ch, rate=2)
        x3 = self.conv_block(x2, int_ch, rate=4)

        """ Bridge """
        x4 = self.conv_block(x3, int_ch, rate=8)

        """ Decoder """
        x = Concatenate()([x4, x3])
        x = self.conv_block(x, int_ch, rate=4)

        x = Concatenate()([x, x2])
        x = self.conv_block(x, int_ch, rate=2)

        x = Concatenate()([x, x1])
        x = self.conv_block(x, out_ch, rate=1)

        """ Addition """
        x = Add()([x, x0])
        return x
    
    def u2net(self,input_shape, out_ch, int_ch, num_classes):
        """ Input Layer """
        inputs = Input(input_shape)
        s0 = inputs

        """ Encoder """
        s1 = self.RSU_L(s0, out_ch[0], int_ch[0], 7)
        p1 = MaxPool2D((2, 2))(s1)

        s2 = self.RSU_L(p1, out_ch[1], int_ch[1], 6)
        p2 = MaxPool2D((2, 2))(s2)

        s3 = self.RSU_L(p2, out_ch[2], int_ch[2], 5)
        p3 = MaxPool2D((2, 2))(s3)

        s4 = self.RSU_L(p3, out_ch[3], int_ch[3], 4)
        p4 = MaxPool2D((2, 2))(s4)

        s5 = self.RSU_4F(p4, out_ch[4], int_ch[4])
        p5 = MaxPool2D((2, 2))(s5)

        """ Bridge """
        b1 = self.RSU_4F(p5, out_ch[5], int_ch[5])
        b2 = UpSampling2D(size=(2, 2), interpolation="bilinear")(b1)

        """ Decoder """
        d1 = Concatenate()([b2, s5])
        d1 = self.RSU_4F(d1, out_ch[6], int_ch[6])
        u1 = UpSampling2D(size=(2, 2), interpolation="bilinear")(d1)

        d2 = Concatenate()([u1, s4])
        d2 = self.RSU_L(d2, out_ch[7], int_ch[7], 4)
        u2 = UpSampling2D(size=(2, 2), interpolation="bilinear")(d2)

        d3 = Concatenate()([u2, s3])
        d3 = self.RSU_L(d3, out_ch[8], int_ch[8], 5)
        u3 = UpSampling2D(size=(2, 2), interpolation="bilinear")(d3)

        d4 = Concatenate()([u3, s2])
        d4 = self.RSU_L(d4, out_ch[9], int_ch[9], 6)
        u4 = UpSampling2D(size=(2, 2), interpolation="bilinear")(d4)

        d5 = Concatenate()([u4, s1])
        d5 = self.RSU_L(d5, out_ch[10], int_ch[10], 7)

        """ Side Outputs """
        y1 = Conv2D(num_classes, 3, padding="same")(d5)

        y2 = Conv2D(num_classes, 3, padding="same")(d4)
        y2 = UpSampling2D(size=(2, 2), interpolation="bilinear")(y2)

        y3 = Conv2D(num_classes, 3, padding="same")(d3)
        y3 = UpSampling2D(size=(4, 4), interpolation="bilinear")(y3)

        y4 = Conv2D(num_classes, 3, padding="same")(d2)
        y4 = UpSampling2D(size=(8, 8), interpolation="bilinear")(y4)

        y5 = Conv2D(num_classes, 3, padding="same")(d1)
        y5 = UpSampling2D(size=(16, 16), interpolation="bilinear")(y5)

        y6 = Conv2D(num_classes, 3, padding="same")(b1)
        y6 = UpSampling2D(size=(32, 32), interpolation="bilinear")(y6)

        y0 = Concatenate()([y1, y2, y3, y4, y5, y6])
        y0 = Conv2D(num_classes, 3, padding="same")(y0)

        y0 = Activation("sigmoid", name="y0")(y0)
        y1 = Activation("sigmoid", name="y1")(y1)
        y2 = Activation("sigmoid", name="y2")(y2)
        y3 = Activation("sigmoid", name="y3")(y3)
        y4 = Activation("sigmoid", name="y4")(y4)
        y5 = Activation("sigmoid", name="y5")(y5)
        y6 = Activation("sigmoid", name="y6")(y6)

        model = tf.keras.models.Model(inputs, outputs=[y0, y1, y2, y3, y4, y5, y6])
        return model
    
    def get_base_model(self, params_filepath = PARAMS_FILE_PATH):
        params_file_path = params_filepath
        params = read_yaml(params_file_path)
        model = self.u2net(params.IMAGE_SIZE, params.OUT_CH, params.INT_CH, num_classes=params.CLASSES)
        model.compile(loss="binary_crossentropy", optimizer=Adam(params.LEARNING_RATE))
        return model
    
    
    

In [42]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
#     model = prepare_base_model.get_base_model()
#     model = prepare_base_model.u2net(params.IMAGE_SIZE, params.OUT_CH, params.INT_CH, num_classes=1)
#     model.compile(loss="binary_crossentropy", optimizer="adam")   
    model = prepare_base_model.get_base_model()
    model.summary()

except Exception as e:
    raise e


[2023-08-29 16:49:01,688: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-29 16:49:01,700: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-29 16:49:01,704: INFO: common: created directory at: artifacts]
[2023-08-29 16:49:01,711: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-08-29 16:49:01,721: INFO: common: yaml file: params.yaml loaded successfully]


NameError: name 'params_INT_CH' is not defined

In [34]:
config_model = read_yaml(CONFIG_FILE_PATH)
save_model_path = config_model.prepare_base_model.base_model_path
print(save_model_path)

[2023-08-29 16:04:17,644: INFO: common: yaml file: config/config.yaml loaded successfully]
artifacts/prepare_base_model/base_model.h5


In [35]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
#     model = prepare_base_model.get_base_model()
    model = prepare_base_model.u2net(params.IMAGE_SIZE, params.OUT_CH, params.INT_CH, num_classes=1)
    model.compile(loss="binary_crossentropy", optimizer=Adam(params.LEARNING_RATE))   
    model.summary()
    model.save(save_model_path)

except Exception as e:
    raise e


[2023-08-29 16:04:25,576: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-29 16:04:25,587: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-29 16:04:25,592: INFO: common: created directory at: artifacts]
[2023-08-29 16:04:25,598: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_493 (Conv2D)            (None, 512, 512, 64  1792        ['input_10[0][0]']               
                                )                

 batch_normalization_472 (Batch  (None, 16, 16, 32)  128         ['conv2d_500[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_471 (Activation)    (None, 16, 16, 32)   0           ['batch_normalization_472[0][0]']
                                                                                                  
 concatenate_210 (Concatenate)  (None, 16, 16, 64)   0           ['activation_471[0][0]',         
                                                                  'activation_470[0][0]']         
                                                                                                  
 conv2d_501 (Conv2D)            (None, 16, 16, 32)   18464       ['concatenate_210[0][0]']        
                                                                                                  
 batch_nor

 activation_477 (Activation)    (None, 512, 512, 64  0           ['batch_normalization_478[0][0]']
                                )                                                                 
                                                                                                  
 add_45 (Add)                   (None, 512, 512, 64  0           ['activation_477[0][0]',         
                                )                                 'activation_464[0][0]']         
                                                                                                  
 max_pooling2d_142 (MaxPooling2  (None, 256, 256, 64  0          ['add_45[0][0]']                 
 D)                             )                                                                 
                                                                                                  
 conv2d_507 (Conv2D)            (None, 256, 256, 12  73856       ['max_pooling2d_142[0][0]']      
          

                                                                                                  
 activation_485 (Activation)    (None, 16, 16, 32)   0           ['batch_normalization_486[0][0]']
                                                                                                  
 up_sampling2d_162 (UpSampling2  (None, 32, 32, 32)  0           ['activation_485[0][0]']         
 D)                                                                                               
                                                                                                  
 concatenate_217 (Concatenate)  (None, 32, 32, 64)   0           ['up_sampling2d_162[0][0]',      
                                                                  'activation_482[0][0]']         
                                                                                                  
 conv2d_515 (Conv2D)            (None, 32, 32, 32)   18464       ['concatenate_217[0][0]']        
          

 max_pooling2d_148 (MaxPooling2  (None, 64, 64, 64)  0           ['activation_491[0][0]']         
 D)                                                                                               
                                                                                                  
 conv2d_521 (Conv2D)            (None, 64, 64, 64)   36928       ['max_pooling2d_148[0][0]']      
                                                                                                  
 batch_normalization_493 (Batch  (None, 64, 64, 64)  256         ['conv2d_521[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_492 (Activation)    (None, 64, 64, 64)   0           ['batch_normalization_493[0][0]']
                                                                                                  
 max_pooli

...

                                                                                                  
 batch_normalization_550 (Batch  (None, 128, 128, 12  512        ['conv2d_578[0][0]']             
 Normalization)                 8)                                                                
                                                                                                  
 activation_549 (Activation)    (None, 128, 128, 12  0           ['batch_normalization_550[0][0]']
                                8)                                                                
                                                                                                  
 add_53 (Add)                   (None, 128, 128, 12  0           ['activation_549[0][0]',         
                                8)                                'activation_540[0][0]']         
                                                                                                  
 up_sampli

 concatenate_248 (Concatenate)  (None, 16, 16, 64)   0           ['activation_556[0][0]',         
                                                                  'activation_555[0][0]']         
                                                                                                  
 conv2d_586 (Conv2D)            (None, 16, 16, 32)   18464       ['concatenate_248[0][0]']        
                                                                                                  
 batch_normalization_558 (Batch  (None, 16, 16, 32)  128         ['conv2d_586[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_557 (Activation)    (None, 16, 16, 32)   0           ['batch_normalization_558[0][0]']
                                                                                                  
 up_sampli

                                                                                                  
 conv2d_592 (Conv2D)            (None, 512, 512, 16  9232        ['activation_562[0][0]']         
                                )                                                                 
                                                                                                  
 batch_normalization_564 (Batch  (None, 512, 512, 16  64         ['conv2d_592[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 activation_563 (Activation)    (None, 512, 512, 16  0           ['batch_normalization_564[0][0]']
                                )                                                                 
                                                                                                  
 max_pooli

                                                                                                  
 concatenate_255 (Concatenate)  (None, 32, 32, 32)   0           ['up_sampling2d_185[0][0]',      
                                                                  'activation_567[0][0]']         
                                                                                                  
 conv2d_600 (Conv2D)            (None, 32, 32, 16)   4624        ['concatenate_255[0][0]']        
                                                                                                  
 batch_normalization_572 (Batch  (None, 32, 32, 16)  64          ['conv2d_600[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_571 (Activation)    (None, 32, 32, 16)   0           ['batch_normalization_572[0][0]']
          

                                                                                                  
 up_sampling2d_190 (UpSampling2  (None, 512, 512, 1)  0          ['conv2d_606[0][0]']             
 D)                                                                                               
                                                                                                  
 up_sampling2d_191 (UpSampling2  (None, 512, 512, 1)  0          ['conv2d_607[0][0]']             
 D)                                                                                               
                                                                                                  
 up_sampling2d_192 (UpSampling2  (None, 512, 512, 1)  0          ['conv2d_608[0][0]']             
 D)                                                                                               
                                                                                                  
 up_sampli